### Useful

In [1]:
%%bash 
uname -a 
date
pwd

Darwin Air-de-j.lan 20.2.0 Darwin Kernel Version 20.2.0: Wed Dec  2 20:39:59 PST 2020; root:xnu-7195.60.75~1/RELEASE_X86_64 x86_64
Fri May  6 08:21:18 CEST 2022
/Users/poderozita/z2022_1/REPOSITORY/NVCC


## Challenge 

In [2]:
%%writefile smallest.cu 
#include <stdio.h>

__global__
void cuda_hello()
{
  printf(">> [smallest] Hello from GPU!\n");
}

int main()
{
    printf(">> [smallest] Hello from CPU!\n");   
    cuda_hello<<<1,1>>>();
    return 0;
}

Overwriting smallest.cu


In [3]:
%%writefile CMakeLists.txt 

cmake_minimum_required(VERSION 3.8 FATAL_ERROR)
project(hello LANGUAGES CXX CUDA)
enable_language(CUDA)
add_executable(smallest.x ../smallest.cu)

## CMAKE_CUDA_ARCHITECTURES now detected for NVCC, empty CUDA_ARCHITECTURES not allowed.
set_property(TARGET smallest.x PROPERTY CUDA_ARCHITECTURES OFF) 

enable_testing()
add_test(NAME smallest COMMAND smallest.x) 

Overwriting CMakeLists.txt


## A solution 

In [4]:
%%writefile GET_NVCC.sh 

## https://docs.nvidia.com/cuda/cuda-installation-guide-linux/index.html
SETUP_NVCC()
{ 
  echo "wget..."
  wget https://developer.download.nvidia.com/compute/cuda/redist/cuda_nvcc/linux-x86_64/cuda_nvcc-linux-x86_64-${RELEASE}-archive.tar.xz
  
  echo "tar..."
  rm -rf cuda_nvcc-linux-x86_64-${RELEASE}-archive
  tar -xf cuda_nvcc-linux-x86_64-${RELEASE}-archive.tar.xz 
}


SETUP_CUDART()
{ 
  echo "wget..."
  wget https://developer.download.nvidia.com/compute/cuda/redist/cuda_cudart/linux-x86_64/cuda_cudart-linux-x86_64-${RELEASE}-archive.tar.xz
  
  echo "tar..."
  rm -rf cuda_cudart-linux-x86_64-${RELEASE}-archive
  tar -xf cuda_cudart-linux-x86_64-${RELEASE}-archive.tar.xz
}


INIT()
{
  ## 1. SETUP 
  g++ --version   ## g++ (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
  cmake --version ## cmake version 3.16.3

  NVCC_PATH=${PWD}/cuda_nvcc-linux-x86_64-${RELEASE}-archive
  CUDART_PATH=${PWD}/cuda_cudart-linux-x86_64-${RELEASE}-archive

  NVCC=$NVCC_PATH/bin/nvcc
  $NVCC --version

  rm -rf BUILD
  mkdir BUILD 
  cd BUILD 

  ## 2. COMPILATION
  ## 2.A. SIMPLEST  
  $NVCC -o smallest.x ../smallest.cu -I$CUDART_PATH/include -L$CUDART_PATH/lib
  ./smallest.x 
  rm smallest.x  

  ## 2.B. EASIEST  
  export CPLUS_INCLUDE_PATH=$CPLUS_INCLUDE_PATH:$CUDART_PATH/include 
  export LIBRARY_PATH=$LIBRARY_PATH:$CUDART_PATH/lib 
  export CUDACXX=$NVCC 

  $CUDACXX  -o smallest.x ../smallest.cu  
  ./smallest.x 
  rm smallest.x
    
  ## 2.C. FANCIEST (https://www.collinsdictionary.com/dictionary/english/fanciest)
  rm -rf CMakeCache.txt CMakeFiles  
  cmake ..
  make
  ctest
}


RELEASE=11.5.50 
SETUP_NVCC
SETUP_CUDART
INIT


Overwriting GET_NVCC.sh


In [ ]:
#!bash GET_NVCC.sh ## uncomment

## Containerization

In [5]:
%%writefile Dockerfile.ci 

FROM ubuntu:focal 
RUN apt-get --yes -qq update 
RUN DEBIAN_FRONTEND=noninteractive TZ=Etc/UTC apt-get -y install tzdata
RUN apt-get --yes -qq update 

RUN apt-get --yes -qq install wget xz-utils
RUN apt-get --yes -qq install build-essential  
RUN apt-get --yes -qq install gcc g++  
RUN apt-get --yes -qq install cmake 
RUN apt-get --yes -qq install cmake-curses-gui ## ccmake 
RUN apt-get --yes -qq clean
RUN rm -rf /var/lib/apt/lists/*

Overwriting Dockerfile.ci


In [6]:
%%writefile workflow.sh 


DOCKER_BUILD()
{
  docker build . \
    --file Dockerfile.ci \
    --tag $IMAGE_NAME  \
    > /dev/null 
}


DOCKER_IMAGES()
{
  IMAGE_ID=$(docker images --filter=reference=$IMAGE_NAME --format "{{.ID}}") 
  echo "IMAGE_ID:", $IMAGE_ID    
}


DOCKER_RUN()
{
  docker run \
    --rm \
    --name ${CONTAINER_NAME} \
    --volume ${PWD}:/home \
    --workdir /home \
    -i ${IMAGE_NAME} \
    ${TOBEEXECUTED}
}


CONTAINER_NAME=nvcc_container
IMAGE_NAME=nvcc_image 

DOCKER_BUILD 

TOBEEXECUTED="ls -la"
DOCKER_RUN

TOBEEXECUTED="pwd"
DOCKER_RUN

TOBEEXECUTED="bash GET_NVCC.sh"
DOCKER_RUN

#
# docker run --rm --name nvcc_container --volume ${PWD}:/home --workdir /home -i nvcc_image bash GET_NVCC.sh
# 

Overwriting workflow.sh


In [7]:
#!bash workflow.sh 

[+] Building 0.0s (0/1)                                                         
[+] Building 0.2s (2/2)                                                         
 => [internal] load build definition from Dockerfile.ci                    0.0s
 => => transferring dockerfile: 487B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
[+] Building 0.3s (2/3)                                                         
 => [internal] load build definition from Dockerfile.ci                    0.0s
 => => transferring dockerfile: 487B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/ubuntu:focal            0.1s
[+] Building 0.5s (2/3)              


Use 'docker scan' to run Snyk tests against images to find vulnerabilities and learn how to fix them
total 111468
drwxr-xr-x 24 root  root      768 May  6 06:18 .
drwxr-xr-x  1 root  root     4096 May  6 06:21 ..
-rw-r--r--  1 root  root     6148 May  5 04:41 .DS_Store
drwxr-xr-x  4 root  root      128 May  6 04:21 .ipynb_checkpoints
drwxr-xr-x  9 root  root      288 May  6 06:19 BUILD
-rw-r--r--  1 root  root      364 May  6 06:21 CMakeLists.txt
-rw-r--r--  1 root  root      525 May  5 08:58 CMakeLists.txt1
-rw-r--r--  1 root  root      440 May  6 06:21 Dockerfile.ci
-rw-r--r--  1 root  root     1637 May  6 06:21 GET_NVCC.sh
-rw-r--r--  1 root  root     3588 May  5 08:50 Makefile
-rwxr-xr-x  1 root  root   804672 May  5 07:53 a.out
-rw-r--r--  1 root  root     1456 May  5 08:33 cmake_install.cmake
drwxr-xr-x  6 25503 tape      192 Sep 14  2021 cuda_cudart-linux-x86_64-11.5.50-archive
-rw-r--r--  1 root  root   835692 Oct 16  2021 cuda_cudart-linux-x86_64-11.5.50-archive.tar.xz
-rw-r-

  3750K .......... .......... .......... .......... .......... 10% 1.03M 32s
  3800K .......... .......... .......... .......... .......... 10% 1.10M 32s
  3850K .......... .......... .......... .......... .......... 10% 1.20M 31s
  3900K .......... .......... .......... .......... .......... 11%  848K 31s
  3950K .......... .......... .......... .......... .......... 11%  831K 31s
  4000K .......... .......... .......... .......... .......... 11% 1011K 31s
  4050K .......... .......... .......... .......... .......... 11% 1.21M 31s
  4100K .......... .......... .......... .......... .......... 11% 1.53M 31s
  4150K .......... .......... .......... .......... .......... 11% 1.23M 31s
  4200K .......... .......... .......... .......... .......... 11% 1.53M 31s
  4250K .......... .......... .......... .......... .......... 12% 1.65M 31s
  4300K .......... .......... .......... .......... .......... 12% 1.44M 30s
  4350K .......... .......... .......... .......... .......... 12% 1.36M 30s

  9100K .......... .......... .......... .......... .......... 25% 1010K 27s
  9150K .......... .......... .......... .......... .......... 25%  870K 27s
  9200K .......... .......... .......... .......... .......... 26% 1.32M 27s
  9250K .......... .......... .......... .......... .......... 26% 1.19M 27s
  9300K .......... .......... .......... .......... .......... 26% 1.20M 27s
  9350K .......... .......... .......... .......... .......... 26%  924K 27s
  9400K .......... .......... .......... .......... .......... 26% 1.03M 26s
  9450K .......... .......... .......... .......... .......... 26% 1.38M 26s
  9500K .......... .......... .......... .......... .......... 26% 1.10M 26s
  9550K .......... .......... .......... .......... .......... 27%  684K 26s
  9600K .......... .......... .......... .......... .......... 27% 1.00M 26s
  9650K .......... .......... .......... .......... .......... 27%  996K 26s
  9700K .......... .......... .......... .......... .......... 27%  962K 26s

 14400K .......... .......... .......... .......... .......... 40% 1.04M 22s
 14450K .......... .......... .......... .......... .......... 40%  936K 22s
 14500K .......... .......... .......... .......... .......... 41% 1002K 22s
 14550K .......... .......... .......... .......... .......... 41%  917K 22s
 14600K .......... .......... .......... .......... .......... 41%  762K 22s
 14650K .......... .......... .......... .......... .......... 41%  745K 22s
 14700K .......... .......... .......... .......... .......... 41%  946K 22s
 14750K .......... .......... .......... .......... .......... 41%  651K 22s
 14800K .......... .......... .......... .......... .......... 41%  950K 22s
 14850K .......... .......... .......... .......... .......... 42% 1.14M 22s
 14900K .......... .......... .......... .......... .......... 42% 1.01M 22s
 14950K .......... .......... .......... .......... .......... 42%  966K 22s
 15000K .......... .......... .......... .......... .......... 42%  906K 22s

 19700K .......... .......... .......... .......... .......... 55% 1.22M 17s
 19750K .......... .......... .......... .......... .......... 55%  893K 17s
 19800K .......... .......... .......... .......... .......... 55%  946K 17s
 19850K .......... .......... .......... .......... .......... 56% 1.13M 17s
 19900K .......... .......... .......... .......... .......... 56% 1011K 16s
 19950K .......... .......... .......... .......... .......... 56%  635K 16s
 20000K .......... .......... .......... .......... .......... 56%  808K 16s
 20050K .......... .......... .......... .......... .......... 56%  879K 16s
 20100K .......... .......... .......... .......... .......... 56%  977K 16s
 20150K .......... .......... .......... .......... .......... 56%  963K 16s
 20200K .......... .......... .......... .......... .......... 57%  975K 16s
 20250K .......... .......... .......... .......... .......... 57%  893K 16s
 20300K .......... .......... .......... .......... .......... 57%  932K 16s

 25000K .......... .......... .......... .......... .......... 70% 1.12M 11s
 25050K .......... .......... .......... .......... .......... 70% 1.15M 11s
 25100K .......... .......... .......... .......... .......... 70% 1.17M 11s
 25150K .......... .......... .......... .......... .......... 71%  824K 11s
 25200K .......... .......... .......... .......... .......... 71%  830K 11s
 25250K .......... .......... .......... .......... .......... 71% 1.15M 11s
 25300K .......... .......... .......... .......... .......... 71% 1005K 11s
 25350K .......... .......... .......... .......... .......... 71% 1.02M 11s
 25400K .......... .......... .......... .......... .......... 71% 1.21M 11s
 25450K .......... .......... .......... .......... .......... 71% 1.32M 11s
 25500K .......... .......... .......... .......... .......... 72% 1.32M 11s
 25550K .......... .......... .......... .......... .......... 72% 1.08M 10s
 25600K .......... .......... .......... .......... .......... 72% 1.40M 10s

 30350K .......... .......... .......... .......... .......... 85%  769K 5s
 30400K .......... .......... .......... .......... .......... 85%  882K 5s
 30450K .......... .......... .......... .......... .......... 86%  796K 5s
 30500K .......... .......... .......... .......... .......... 86%  885K 5s
 30550K .......... .......... .......... .......... .......... 86%  672K 5s
 30600K .......... .......... .......... .......... .......... 86%  756K 5s
 30650K .......... .......... .......... .......... .......... 86%  754K 5s
 30700K .......... .......... .......... .......... .......... 86%  806K 5s
 30750K .......... .......... .......... .......... .......... 86%  885K 5s
 30800K .......... .......... .......... .......... .......... 87%  975K 5s
 30850K .......... .......... .......... .......... .......... 87%  948K 5s
 30900K .......... .......... .......... .......... .......... 87%  701K 5s
 30950K .......... .......... .......... .......... .......... 87%  652K 5s
 31000K ....


     0K .......... .......... .......... .......... ..........  6%  981K 1s
    50K .......... .......... .......... .......... .......... 12%  992K 1s
   100K .......... .......... .......... .......... .......... 18% 1.02M 1s
   150K .......... .......... .......... .......... .......... 24%  994K 1s
   200K .......... .......... .......... .......... .......... 30% 1.08M 1s
   250K .......... .......... .......... .......... .......... 36% 1.15M 0s
   300K .......... .......... .......... .......... .......... 42% 1.18M 0s
   350K .......... .......... .......... .......... .......... 49%  945K 0s
   400K .......... .......... .......... .......... .......... 55% 1.48M 0s
   450K .......... .......... .......... .......... .......... 61% 1.20M 0s
   500K .......... .......... .......... .......... .......... 67% 1.26M 0s
   550K .......... .......... .......... .......... .......... 73%  989K 0s
   600K .......... .......... .......... .......... .......... 79% 1.14M 0s
   650K ...

## Done!!